In [2]:
# ✅ Environment Setup
!pip install -U bitsandbytes transformers accelerate peft faiss-cpu pymupdf fitz sentence-transformers langchain --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
!pip install --upgrade --force-reinstall pymupdf

  Using cached pymupdf-1.25.5-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.4 kB)
Using cached pymupdf-1.25.5-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (20.0 MB)
  Attempting uninstall: pymupdf
    Found existing installation: PyMuPDF 1.25.5
    Uninstalling PyMuPDF-1.25.5:
      Successfully uninstalled PyMuPDF-1.25.5


In [4]:
# ✅ Imports
import torch
import fitz
import faiss
import numpy as np
from huggingface_hub import login
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [5]:
# ✅ Hugging Face Token Login
login(token="")


In [6]:
# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/Models/18XAZpERgrl5A004d0oZqEYhvY95Jw_eZ"
!ls "/content/drive/MyDrive/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'10a labs Shreyash_Kakde_Cover Letter.pdf'   pubmed_json
 20241015_212539.jpg			     PUBMED_SAMPLED
 Biomedical_QnA_Chatbot.mp4		     S25-block6-GNN.gdoc
'Colab Notebooks'			     ScalpCare_Demo_Video.mp4
 Data					     Shreyash_Resume.pdf
 DL					     Trichoscopy_Report.pdf
 LA					    'Untitled document (1).gdoc'
 MERGED_DATASET				    'Untitled document.gdoc'
 mistral-lora-checkpoint		    'USC_Shreyash_Resume (1).pdf'
 ML_Research				    'USC_Shreyash_Resume (2).pdf'
 OUTPUT					     USC_Shreyash_Resume.pdf
 PREVIOUS_CHECKPOINT			     Wiki_data
 PubMed					     WIKI_SAMPLED


In [7]:
# ✅ PDF Text Extraction
doc = fitz.open("/content/medical_book.pdf")
text = "".join([page.get_text() for page in doc])

with open("medical_book.txt", "w") as f:
    f.write(text)

print(f"✅ Extracted {len(text)} characters from the book.")


✅ Extracted 3135093 characters from the book.


In [8]:
# ✅ Text Chunking
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = splitter.create_documents([text])
split_texts = [doc.page_content for doc in split_docs]

print(f"✅ Created {len(split_texts)} chunks for embedding.")


✅ Created 3912 chunks for embedding.


In [9]:
# ✅ Embedding and FAISS Indexing
embedder = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
doc_embeddings = embedder.encode(split_texts, convert_to_numpy=True, show_progress_bar=True)

dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

print(f"✅ FAISS index created with dimension {dimension} and {index.ntotal} vectors.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/123 [00:00<?, ?it/s]

✅ FAISS index created with dimension 384 and 3912 vectors.


In [10]:
# Log in to your Hugging Face account
login(token="") # replace with your token from the settings page on huggingface

import os
os.makedirs("/content/offload", exist_ok=True)

# Load tokenizer from base model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.3")

# Define BitsAndBytes configuration for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load base model with 4-bit quantization and device_map="auto"
# This is generally more memory efficient than loading in FP16 first
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.3",
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16 # Keep this for compute dtype compatibility
)

# Load QLoRA adapter weights on top of the quantized base model
model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/MyDrive/mistral-lora-checkpoint/mistral-lora-checkpoint",
    # offload_folder is less critical here as 4-bit quantization is used
    # but you can keep it if you think it helps with adapter loading
    offload_folder="/content/offload"
)

model.eval()

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.l

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pro

In [25]:
def rag_chat(query, embedder, index, split_texts, model, tokenizer, top_k=5):
    query_embedding = embedder.encode([query])
    _, indices = index.search(query_embedding, top_k)
    context = "\n\n".join([split_texts[i] for i in indices[0]])

    prompt = f"""You are a concise and factual biomedical assistant.

    Use the following context to answer the question in 3 complete, non-repetitive sentences, forming a single paragraph.
    Absolutely do NOT use bullet points, lists, or numbered items.
    Keep your answer brief and under 60 words.

    Context:
    {context}

    Question: {query}
    Answer:"""


    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=200)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the portion after 'Answer:'
    if "Answer:" in decoded:
        return decoded.split("Answer:")[-1].strip()
    else:
        return decoded.strip()

In [12]:
!pip install bert-score
bert_f1_scores = []


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.2 MB/s eta 0:00:00


In [13]:
# Example biomedical query
response = rag_chat("What are the symptoms and impact of glaucoma?", embedder, index, split_texts, model, tokenizer)
print("🧠 Response:", response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧠 Response: Glaucoma is a disease of the eye marked by increased
pressure within the eyeball that can result in damage
to the optic disk and gradual loss of vision. It is
usually associated with increased IOP, but that is not
always the case. The three factors associated with
glaucoma are increased IOP, a change in the optic
nerve head, and changes in the visual field.
    Symptoms of glaucoma include decreased visual
field and blindness if not treated. It is usually
associated with increased IOP, but that is not always
the case. The three factors associated with glaucoma
are increased IOP, a change in the optic nerve head,
and changes in the visual field.
    Glaucoma is a disease of the eye marked by increased
pressure within the eyeball that can result in damage
to the optic


In [18]:
from bert_score import score

# Example: prediction vs reference
prediction = "Glaucoma is a disease of the eye marked by increased pressure within the eyeball that can result in damage to the optic disk and gradual loss of vision. It is usually associated with increased IOP, but that is notalways the case. The three factors associated with glaucoma are increased IOP, a change in the optic nerve head, and changes in the visual field. Symptoms of glaucoma include decreased visual field and blindness if not treated. It is usually associated with increased IOP, but that is not always the case. The three factors associated with glaucoma are increased IOP, a change in the optic nerve head, and changes in the visual field. Glaucoma is a disease of the eye marked by increased pressure within the eyeball that can result in damage to the optic"
reference = "Glaucoma can cause gradual vision loss starting with peripheral vision. If untreated, it can lead to irreversible blindness due to optic nerve damage."
P, R, F1 = score([prediction], [reference], lang="en", verbose=True)
# Append just the F1 score
bert_f1_scores.append({
    "Question": "Q2",  # Change to Q2, Q3, etc. in each cell
    "BERTScore F1": round(F1.item(), 4)
})

print(f"✅ Q2 F1: {F1.item():.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.12 seconds, 8.34 sentences/sec
✅ Q2 F1: 0.8641


In [ ]:
!pip install gradio
import gradio as gr
# Gradio Interface
def process_query(query):
    if not query:
        return "Please enter a medical question."

    return rag_chat(query,embedder, index, split_texts, model, tokenizer)

# Create the Gradio interface
demo = gr.Interface(
    fn=process_query,
    inputs=gr.Textbox(lines=2, placeholder="Enter your medical question here..."),
    outputs=gr.Textbox(label="Medical Assistant Response"),
    title="Medical RAG Chatbot",
    description="Ask medical questions.",
    examples=[
        ["What are the symptoms and impact of glaucoma?"],
        ["How does the heart's electrical conduction system work?"],
        ["What are the complications of diabetes mellitus?"],
        ["Explain the mechanisms of antibiotic resistance."],
        ["What is the pathophysiology of Alzheimer's disease?"]
    ],
    allow_flagging="never",
    theme=gr.themes.Soft()
)

# Launch the interface
demo.launch(debug=True, share=True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9494e8ac05e6156db7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
